In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [3]:
# Create a dictionary that maps the run# to host_density { run#: host_density }

paramfile = "/home/hill/PyCharmProjects/tickABM/data/host-density/new-model-agg/density-new.2020.Jul.11.18_24_35"
paramd = {}
with open(paramfile, 'r') as file:
    for line in file:
        result = line.replace("\t",",").split(',')
        # attempt to add the density value to run, if it doesn't exist, create it
        try:
            paramd[int(result[0])].append(float(result[4]))
        except KeyError:
            paramd[int(result[0])] = float(result[4])  # { run#: host_density }

for k, v in paramd.items():
    print(k,v)

1 0.1
2 0.1
3 0.1
4 0.1
5 0.1
6 0.1
7 0.1
8 0.1
9 0.1
10 0.1
11 0.4
12 0.4
13 0.4
14 0.4
15 0.4
16 0.4
17 0.4
18 0.4
19 0.4
20 0.4
21 0.7
22 0.7
23 0.7
24 0.7
25 0.7
26 0.7
27 0.7
28 0.7
29 0.7
30 0.7
31 1.0
32 1.0
33 1.0
34 1.0
35 1.0
36 1.0
37 1.0
38 1.0
39 1.0
40 1.0
41 3.0
42 3.0
43 3.0
44 3.0
45 3.0
46 3.0
47 3.0
48 3.0
49 3.0
50 3.0


In [4]:
# Create a dictionary that maps the run# to total ixode { run#: cumulative_ixode }
ixode_count_dict = {}
csv_file = "/media/hill/DATA-LINUX/abm-data/host-density/new-model/aggregate-runs/density-new.2020.Jul.11hab_09"
df = pd.read_csv(csv_file, error_bad_lines=False)
for run in df.groupby('run'):
    current_df = run[1]
    ixode_count_dict[run[0]] = len(current_df['name'].unique())

b'Skipping line 4520553: expected 4 fields, saw 6\nSkipping line 4520554: expected 4 fields, saw 6\nSkipping line 4520555: expected 4 fields, saw 6\nSkipping line 4520556: expected 4 fields, saw 6\nSkipping line 4520557: expected 4 fields, saw 6\nSkipping line 4520558: expected 4 fields, saw 6\nSkipping line 4520559: expected 4 fields, saw 6\nSkipping line 4520560: expected 4 fields, saw 6\nSkipping line 4520561: expected 4 fields, saw 6\nSkipping line 4520562: expected 4 fields, saw 6\nSkipping line 4520563: expected 4 fields, saw 6\nSkipping line 4520564: expected 4 fields, saw 6\nSkipping line 4520565: expected 4 fields, saw 6\n'
b'Skipping line 13516221: expected 4 fields, saw 6\nSkipping line 13516222: expected 4 fields, saw 6\nSkipping line 13516223: expected 4 fields, saw 6\nSkipping line 13516224: expected 4 fields, saw 6\nSkipping line 13516225: expected 4 fields, saw 6\nSkipping line 13516226: expected 4 fields, saw 6\nSkipping line 13516227: expected 4 fields, saw 6\nSkippin

In [5]:
# Creating a data frame from the cumulative_ixodes dict generated above
df_final = pd.DataFrame(ixode_count_dict.items(), columns=['run', 'total_ixodes'])
df_final['host_density'] = 0
df_final.dtypes

run             int64
total_ixodes    int64
host_density    int64
dtype: object

In [6]:
# Iterate through param dictionary and add the host density value to the associated run
for key, value in paramd.items():
    for i in range(len(paramd)):
        df_final.loc[df_final['run']==key, 'host_density'] = value

In [20]:
df_test = pd.read_csv("../data/test")
df_test.dtypes

run               int64
host_density    float64
total_ixodes      int64
dtype: object

In [7]:
# Now we have a df with | 'run' | 'total_ixode' | 'host_density' |
# Create a dictionary with { host_density: agg_ixodes }
agg_ixodes_dict = {}
for density in df_final.groupby('host_density'):
    tmp_df = density[1] # density is a tuple, so density[1] is the df we want
    print(density[0])
    agg_ixodes_dict[density[0]] = tmp_df['total_ixodes'].agg('mean')


# in test file - should get 1880.75 for 0.4, 7898.5 for 0.7, and 24917 for 1.0


0.1
0.4
0.7
1.0
3.0


In [8]:
# Create a dataframe from the agg_ixodes_dict that we can then plot
df_agg_final = pd.DataFrame(agg_ixodes_dict.items(), columns=['host_density', 'agg_ixodes'])
df_agg_final.head()

,host_density,agg_ixodes
0,0.1,9029.9
1,0.4,10171.2
2,0.7,7152.9
3,1.0,8363.9
4,3.0,6565.2


In [9]:
matplotlib.use('Qt5Agg')
plt.plot(df_agg_final['host_density'], df_agg_final['agg_ixodes'])
plt.show(block=True)
